In [130]:
# load json file 
import json
import os
import sys

import pandas as pd
import numpy as np

dataset_name = "celeba_20_ids"

paths = [f"output/runs/survey/{dataset_name}.json", f"output/runs/survey/{dataset_name}_seed_1.json", f"output/runs/survey/{dataset_name}_seed_1.json"]

datasets = []
for path in paths: 
    with open(path, 'r') as f:
        data = json.load(f)

    len(data)

    for entry in data:
        try :
            entry['lr'] = entry['parameters']['optimizer']['parameters']['lr']
        except KeyError:
            entry['lr'] = None

    df = pd.DataFrame(data)
    # remove dataset column 
    df = df.drop(columns=['dataset', 'parameters', 'RunTime', 'sklearn.metrics.accuracy_score.test.original', 'sklearn.metrics.accuracy_score.forget.original', 'sklearn.metrics.accuracy_score.retain.original'])

    forget_accuracies = df['sklearn.metrics.accuracy_score.forget.unlearned']
    gold_accuracy = df['sklearn.metrics.accuracy_score.forget.unlearned'][0]
    df['forget_diff'] = np.abs(forget_accuracies - gold_accuracy)

    def nomus(f1, forget_score, l=0.5):
        return l * f1 + (1 - l) * (1-forget_score*2)
    
    df['Unlearning_Score'] = np.abs(df['UMIA'] - 0.5)

    df['nomus'] = nomus(df['sklearn.metrics.accuracy_score.test.unlearned'], df['Unlearning_Score'])

    # Remove relearning score 
    df = df.drop(columns=['RelearnTime'])
    

    
    datasets.append(df)

len(datasets)

3

In [131]:
len(datasets[0]), len(datasets[1]), len(datasets[2])

(36, 36, 36)

In [132]:
datasets[0]

,unlearner,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.retain.unlearned,AUS,AIN,UMIA,lr,forget_diff,Unlearning_Score,nomus
0,GoldModel,0.923653,0.927374,0.938075,0.993613,1.000000,0.508837,NaN,0.000000,0.008837,0.952989
1,AdvancedNegGrad,0.868794,0.031657,0.892745,0.513000,1.995025,0.966977,0.00010,0.895717,0.466977,0.467420
2,AdvancedNegGrad,0.695875,0.402235,0.710111,0.594858,49.756219,0.662791,0.00001,0.525140,0.162791,0.685147
3,BadTeaching,0.519443,0.536313,0.517639,0.583262,1.995025,0.502326,0.00100,0.391061,0.002326,0.757396
4,BadTeaching,0.480557,0.463687,0.482361,0.545020,1.995025,0.490930,0.00010,0.463687,0.009070,0.731209
5,BadTeaching,0.525810,0.545624,0.523759,0.587821,13.935323,0.518372,0.00001,0.381750,0.018372,0.744533
6,Finetuning,0.924037,0.949721,0.939104,0.972712,1.000000,0.528372,0.00100,0.022346,0.028372,0.933647
7,Finetuning,0.923406,0.949721,0.948626,0.971499,1.000000,0.526279,0.00010,0.022346,0.026279,0.935424
8,Finetuning,0.928675,0.968343,0.944400,0.964090,0.004975,0.534419,0.00001,0.040968,0.034419,0.929919
9,Finetuning,0.926315,0.957169,0.939474,0.970043,7.467662,0.537209,0.00100,0.029795,0.037209,0.925948


In [133]:
names = datasets[0].columns
names

Index(['unlearner', 'sklearn.metrics.accuracy_score.test.unlearned',
       'sklearn.metrics.accuracy_score.forget.unlearned',
       'sklearn.metrics.accuracy_score.retain.unlearned', 'AUS', 'AIN', 'UMIA',
       'lr', 'forget_diff', 'Unlearning_Score', 'nomus'],
      dtype='object')

In [134]:
evaluate_mean_std_for = ['sklearn.metrics.accuracy_score.test.unlearned', 'sklearn.metrics.accuracy_score.forget.unlearned', 'sklearn.metrics.accuracy_score.retain.unlearned', 'AUS', 'AIN', 'UMIA', 'forget_diff', 'Unlearning_Score', 'nomus']

new_df = pd.DataFrame(columns=names)

for name in names:
    if name in evaluate_mean_std_for:
        new_df[name] = np.mean([df[name] for df in datasets], axis=0)
        new_df[name + '_std'] = np.std([df[name] for df in datasets], axis=0)
    else:
        new_df[name] = datasets[0][name]

new_df

,unlearner,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.retain.unlearned,AUS,AIN,UMIA,lr,forget_diff,Unlearning_Score,nomus,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS_std,AIN_std,UMIA_std,forget_diff_std,Unlearning_Score_std,nomus_std
0,GoldModel,0.924166,0.936065,0.937864,0.989631,0.778516,0.508837,NaN,0.000000,0.008837,0.953246,0.000362,6.144938e-03,1.487733e-04,0.002816,0.156613,0.000000,0.000000e+00,0.000000,0.000181
1,AdvancedNegGrad,0.888901,0.032899,0.909467,0.520503,1.553159,0.971628,0.00010,0.903166,0.471628,0.472823,0.014218,8.778483e-04,1.182425e-02,0.005305,0.312446,0.003289,5.267090e-03,0.003289,0.003820
2,AdvancedNegGrad,0.731277,0.484171,0.748774,0.649319,38.736015,0.635659,0.00001,0.451893,0.135659,0.729980,0.025033,5.793799e-02,2.733872e-02,0.038509,7.792461,0.019185,5.179305e-02,0.019185,0.031701
3,BadTeaching,0.493519,0.487896,0.494120,0.561275,1.553159,0.512868,0.00100,0.448169,0.012868,0.733891,0.018331,3.423608e-02,1.663027e-02,0.015547,0.312446,0.007455,4.038102e-02,0.007455,0.016621
4,BadTeaching,0.480557,0.463687,0.482361,0.548528,1.553159,0.504264,0.00010,0.472377,0.010310,0.729968,0.000000,5.551115e-17,5.551115e-17,0.002481,0.312446,0.009428,6.144938e-03,0.000877,0.000877
5,BadTeaching,0.569609,0.601490,0.570408,0.626623,6.640678,0.517287,0.00001,0.334575,0.017287,0.767518,0.030971,3.950317e-02,3.298564e-02,0.027438,5.158093,0.000767,3.335823e-02,0.000767,0.016253
6,Finetuning,0.922665,0.953445,0.937631,0.970039,0.778516,0.534109,0.00100,0.017381,0.034109,0.927224,0.000970,2.633545e-03,1.041413e-03,0.001890,0.156613,0.004056,3.511393e-03,0.004056,0.004541
7,Finetuning,0.924101,0.954687,0.949752,0.971609,1.000000,0.529690,0.00010,0.018622,0.029690,0.932361,0.000492,3.511393e-03,7.956140e-04,0.000078,0.000000,0.002412,2.633545e-03,0.002412,0.002166
8,Finetuning,0.927889,0.970826,0.945004,0.963734,0.446841,0.533643,0.00001,0.034761,0.033643,0.930301,0.000556,1.755697e-03,4.269148e-04,0.000251,0.312446,0.000548,4.389241e-03,0.000548,0.000270
9,Finetuning,0.924028,0.968343,0.937905,0.958838,2.491435,0.544806,0.00100,0.032278,0.044806,0.917208,0.001617,7.900634e-03,1.109332e-03,0.007923,3.518723,0.005372,1.755697e-03,0.005372,0.006180


In [135]:
# order columns name of the df 

new_df = new_df[['unlearner', 'lr', 'sklearn.metrics.accuracy_score.test.unlearned', 'sklearn.metrics.accuracy_score.test.unlearned_std', 'sklearn.metrics.accuracy_score.forget.unlearned', 'sklearn.metrics.accuracy_score.forget.unlearned_std', 'sklearn.metrics.accuracy_score.retain.unlearned', 'sklearn.metrics.accuracy_score.retain.unlearned_std', 'AUS', 'AUS_std', 'AIN', 'AIN_std', 'forget_diff', 'forget_diff_std', 'UMIA', 'UMIA_std', 'Unlearning_Score', 'Unlearning_Score_std', 'nomus', 'nomus_std']]

In [136]:
new_df

# remove index 1 and 2 

# change unleraner of index 11, 12, 13 to cfk 
new_df['unlearner'][9] = "cfk"
new_df['unlearner'][10] = "cfk"
new_df['unlearner'][11] = "cfk"

# change lr to number 14,15,16 to 1.00E-07, 1.00E-08, 1.00E-06
new_df['lr'][12] = '1.00E-07'
new_df['lr'][13] = '1.00E-08'
new_df['lr'][14] = '1.00E-06'

# change number 29, 30, 31 to unsir 
new_df['unlearner'][27] = 'unsir'
new_df['unlearner'][28] = 'unsir'
new_df['unlearner'][29] = 'unsir'
new_df['lr'][27] = 0.001
new_df['lr'][28] = 0.01
new_df['lr'][29] = 0.0001

# change number 35,36,37 to salun
new_df['unlearner'][33] = 'salun'
new_df['unlearner'][34] = 'salun'
new_df['unlearner'][35] = 'salun'
new_df['lr'][33] = 0.001
new_df['lr'][34] = 0.01
new_df['lr'][35] = 0.0001

new_df['unlearner'][23] = "ssd"
new_df['unlearner'][24] = "ssd"
new_df['unlearner'][25] = "ssd"
new_df['unlearner'][26] = "ssd"
new_df['lr'][23] = "0.1_20"
new_df['lr'][24] = "0.01_20"
new_df['lr'][25] = "0.1_40"
new_df['lr'][26] = "0.01_40"


new_df

/tmp/ipykernel_316471/347910662.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['unlearner'][9] = "cfk"
/tmp/ipykernel_316471/347910662.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,AIN,AIN_std,forget_diff,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std
0,GoldModel,NaN,0.924166,0.000362,0.936065,6.144938e-03,0.937864,1.487733e-04,0.989631,0.002816,0.778516,0.156613,0.000000,0.000000e+00,0.508837,0.000000,0.008837,0.000000,0.953246,0.000181
1,AdvancedNegGrad,0.0001,0.888901,0.014218,0.032899,8.778483e-04,0.909467,1.182425e-02,0.520503,0.005305,1.553159,0.312446,0.903166,5.267090e-03,0.971628,0.003289,0.471628,0.003289,0.472823,0.003820
2,AdvancedNegGrad,0.00001,0.731277,0.025033,0.484171,5.793799e-02,0.748774,2.733872e-02,0.649319,0.038509,38.736015,7.792461,0.451893,5.179305e-02,0.635659,0.019185,0.135659,0.019185,0.729980,0.031701
3,BadTeaching,0.001,0.493519,0.018331,0.487896,3.423608e-02,0.494120,1.663027e-02,0.561275,0.015547,1.553159,0.312446,0.448169,4.038102e-02,0.512868,0.007455,0.012868,0.007455,0.733891,0.016621
4,BadTeaching,0.0001,0.480557,0.000000,0.463687,5.551115e-17,0.482361,5.551115e-17,0.548528,0.002481,1.553159,0.312446,0.472377,6.144938e-03,0.504264,0.009428,0.010310,0.000877,0.729968,0.000877
5,BadTeaching,0.00001,0.569609,0.030971,0.601490,3.950317e-02,0.570408,3.298564e-02,0.626623,0.027438,6.640678,5.158093,0.334575,3.335823e-02,0.517287,0.000767,0.017287,0.000767,0.767518,0.016253
6,Finetuning,0.001,0.922665,0.000970,0.953445,2.633545e-03,0.937631,1.041413e-03,0.970039,0.001890,0.778516,0.156613,0.017381,3.511393e-03,0.534109,0.004056,0.034109,0.004056,0.927224,0.004541
7,Finetuning,0.0001,0.924101,0.000492,0.954687,3.511393e-03,0.949752,7.956140e-04,0.971609,0.000078,1.000000,0.000000,0.018622,2.633545e-03,0.529690,0.002412,0.029690,0.002412,0.932361,0.002166
8,Finetuning,0.00001,0.927889,0.000556,0.970826,1.755697e-03,0.945004,4.269148e-04,0.963734,0.000251,0.446841,0.312446,0.034761,4.389241e-03,0.533643,0.000548,0.033643,0.000548,0.930301,0.000270
9,cfk,0.001,0.924028,0.001617,0.968343,7.900634e-03,0.937905,1.109332e-03,0.958838,0.007923,2.491435,3.518723,0.032278,1.755697e-03,0.544806,0.005372,0.044806,0.005372,0.917208,0.006180


In [137]:
# create a new dataset that, for each unlearner, has the one with the highest nomus score

best = {}

for i, row in new_df.iterrows():
    if row['unlearner'] not in best:
        best[row['unlearner']] = row
    else:
        if row['nomus'] > best[row['unlearner']]['nomus']:
            best[row['unlearner']] = row

best_df = pd.DataFrame(list(best.values()))
best_df

,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,AIN,AIN_std,forget_diff,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std
0,GoldModel,NaN,0.924166,0.000362,0.936065,0.006145,0.937864,0.000149,0.989631,0.002816,0.778516,0.156613,0.000000,0.000000,0.508837,0.000000,0.008837,0.000000,0.953246,0.000181
2,AdvancedNegGrad,0.00001,0.731277,0.025033,0.484171,0.057938,0.748774,0.027339,0.649319,0.038509,38.736015,7.792461,0.451893,0.051793,0.635659,0.019185,0.135659,0.019185,0.729980,0.031701
5,BadTeaching,0.00001,0.569609,0.030971,0.601490,0.039503,0.570408,0.032986,0.626623,0.027438,6.640678,5.158093,0.334575,0.033358,0.517287,0.000767,0.017287,0.000767,0.767518,0.016253
7,Finetuning,0.0001,0.924101,0.000492,0.954687,0.003511,0.949752,0.000796,0.971609,0.000078,1.000000,0.000000,0.018622,0.002634,0.529690,0.002412,0.029690,0.002412,0.932361,0.002166
10,cfk,0.0001,0.923854,0.001837,0.967722,0.008778,0.937581,0.001223,0.959103,0.009163,4.647323,6.567608,0.031657,0.002634,0.542481,0.000110,0.042481,0.000110,0.919447,0.001028
12,FisherForgetting,1.00E-07,0.851990,0.034140,0.881440,0.029847,0.856818,0.036456,0.902777,0.034580,0.778516,0.156613,0.054624,0.035992,0.536357,0.003070,0.036357,0.003070,0.889638,0.014001
15,eu_k,0.01,0.923983,0.001746,0.970826,0.007023,0.938113,0.001064,0.956468,0.007314,0.501386,0.704368,0.034761,0.000878,0.524264,0.006030,0.024264,0.006030,0.937728,0.005156
16,Identity,NaN,0.922775,0.002523,0.970205,0.005267,0.936366,0.001824,0.954771,0.007138,0.003873,0.000779,0.034140,0.000878,0.544419,0.006578,0.044419,0.006578,0.916969,0.007839
17,NegGrad,0.00001,0.922812,0.002419,0.970205,0.005267,0.936346,0.001843,0.954837,0.006944,0.003873,0.000779,0.034140,0.000878,0.542016,0.005701,0.042016,0.005701,0.919390,0.006910
21,Scrub,0.001,0.919006,0.005136,0.944134,0.000000,0.929357,0.006294,0.971873,0.012296,0.778516,0.156613,0.008070,0.006145,0.529380,0.001096,0.029380,0.001096,0.930123,0.003664


In [138]:
# read txt file with the times, where there is a line for each unlearner as "name time", for three times for the evaluation of mean and std 
times = {}

with open(f"output/runs/survey/time/{dataset_name}.txt", 'r') as f:
    for line in f:
        name, time = line.split(":")
        if name not in times:
            times[name] = []
        times[name].append(float(time))

times


{'GoldModel': [1594.3165845870972, 1617.2314856052399, 1584.8357911109924],
 'AdvancedNegGrad': [276.5432913303375, 276.9710681438446, 279.06874442100525],
 'BadTeaching': [155.62881207466125, 158.64404249191284, 155.7620885372162],
 'Finetuning': [43.43743848800659, 44.081308364868164, 43.23821139335632],
 'cfk': [35.66342067718506, 35.816524267196655, 35.06626892089844],
 'FisherForgetting': [1816.6282725334167,
  1712.6152826690675,
  1788.850764541626],
 'eu_k': [176.16479349136353, 166.61287879943848, 166.23280429840088],
 'NegGrad': [0.3463304042816162, 0.32715773582458496, 0.32717037200927734],
 'Scrub': [68.29331278800964, 64.5500967502594, 65.13795256614685],
 'ssd': [101.25245833396912, 101.70019435882568, 103.1240565776825],
 'unsir': [43.0055150986, 43.1281797886, 42.9365627766],
 'SuccessiveRandomLabels': [40.14802575111389,
  40.88470125198364,
  40.284966707229614],
 'salun': [41.04909634590149, 42.7211003304, 42.5594079494]}

In [139]:
# for each entry, do the mean and std and add to the relative unlearner in the best_df

# add time and time_std to the best_df
best_df['time'] = 0
best_df['time_std'] = 0

for i, row in best_df.iterrows():
    name = row['unlearner']
    if name != 'Identity': 
        best_df.loc[i, 'time'] = np.mean(times[name])
        best_df.loc[i, 'time_std'] = np.std(times[name])
    else:
        best_df.loc[i, 'time'] = 0
        best_df.loc[i, 'time_std'] = 0

best_df

/tmp/ipykernel_316471/1241502917.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1598.7946204344432' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  best_df.loc[i, 'time'] = np.mean(times[name])
/tmp/ipykernel_316471/1241502917.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.599261235541196' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  best_df.loc[i, 'time_std'] = np.std(times[name])


,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,...,forget_diff,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std,time,time_std
0,GoldModel,NaN,0.924166,0.000362,0.936065,0.006145,0.937864,0.000149,0.989631,0.002816,...,0.000000,0.000000,0.508837,0.000000,0.008837,0.000000,0.953246,0.000181,1598.794620,13.599261
2,AdvancedNegGrad,0.00001,0.731277,0.025033,0.484171,0.057938,0.748774,0.027339,0.649319,0.038509,...,0.451893,0.051793,0.635659,0.019185,0.135659,0.019185,0.729980,0.031701,277.527701,1.103588
5,BadTeaching,0.00001,0.569609,0.030971,0.601490,0.039503,0.570408,0.032986,0.626623,0.027438,...,0.334575,0.033358,0.517287,0.000767,0.017287,0.000767,0.767518,0.016253,156.678314,1.391044
7,Finetuning,0.0001,0.924101,0.000492,0.954687,0.003511,0.949752,0.000796,0.971609,0.000078,...,0.018622,0.002634,0.529690,0.002412,0.029690,0.002412,0.932361,0.002166,43.585653,0.359795
10,cfk,0.0001,0.923854,0.001837,0.967722,0.008778,0.937581,0.001223,0.959103,0.009163,...,0.031657,0.002634,0.542481,0.000110,0.042481,0.000110,0.919447,0.001028,35.515405,0.323679
12,FisherForgetting,1.00E-07,0.851990,0.034140,0.881440,0.029847,0.856818,0.036456,0.902777,0.034580,...,0.054624,0.035992,0.536357,0.003070,0.036357,0.003070,0.889638,0.014001,1772.698107,43.972391
15,eu_k,0.01,0.923983,0.001746,0.970826,0.007023,0.938113,0.001064,0.956468,0.007314,...,0.034761,0.000878,0.524264,0.006030,0.024264,0.006030,0.937728,0.005156,169.670159,4.595021
16,Identity,NaN,0.922775,0.002523,0.970205,0.005267,0.936366,0.001824,0.954771,0.007138,...,0.034140,0.000878,0.544419,0.006578,0.044419,0.006578,0.916969,0.007839,0.000000,0.000000
17,NegGrad,0.00001,0.922812,0.002419,0.970205,0.005267,0.936346,0.001843,0.954837,0.006944,...,0.034140,0.000878,0.542016,0.005701,0.042016,0.005701,0.919390,0.006910,0.333553,0.009035
21,Scrub,0.001,0.919006,0.005136,0.944134,0.000000,0.929357,0.006294,0.971873,0.012296,...,0.008070,0.006145,0.529380,0.001096,0.029380,0.001096,0.930123,0.003664,65.993787,1.643625


In [140]:
# evaluate speedup with respect to gold model 

gold_time = best_df[best_df['unlearner'] == 'GoldModel']['time'].values[0]
best_df['speedup'] = gold_time / best_df['time']
best_df

,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,...,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std,time,time_std,speedup
0,GoldModel,NaN,0.924166,0.000362,0.936065,0.006145,0.937864,0.000149,0.989631,0.002816,...,0.000000,0.508837,0.000000,0.008837,0.000000,0.953246,0.000181,1598.794620,13.599261,1.000000
2,AdvancedNegGrad,0.00001,0.731277,0.025033,0.484171,0.057938,0.748774,0.027339,0.649319,0.038509,...,0.051793,0.635659,0.019185,0.135659,0.019185,0.729980,0.031701,277.527701,1.103588,5.760847
5,BadTeaching,0.00001,0.569609,0.030971,0.601490,0.039503,0.570408,0.032986,0.626623,0.027438,...,0.033358,0.517287,0.000767,0.017287,0.000767,0.767518,0.016253,156.678314,1.391044,10.204313
7,Finetuning,0.0001,0.924101,0.000492,0.954687,0.003511,0.949752,0.000796,0.971609,0.000078,...,0.002634,0.529690,0.002412,0.029690,0.002412,0.932361,0.002166,43.585653,0.359795,36.681672
10,cfk,0.0001,0.923854,0.001837,0.967722,0.008778,0.937581,0.001223,0.959103,0.009163,...,0.002634,0.542481,0.000110,0.042481,0.000110,0.919447,0.001028,35.515405,0.323679,45.016934
12,FisherForgetting,1.00E-07,0.851990,0.034140,0.881440,0.029847,0.856818,0.036456,0.902777,0.034580,...,0.035992,0.536357,0.003070,0.036357,0.003070,0.889638,0.014001,1772.698107,43.972391,0.901899
15,eu_k,0.01,0.923983,0.001746,0.970826,0.007023,0.938113,0.001064,0.956468,0.007314,...,0.000878,0.524264,0.006030,0.024264,0.006030,0.937728,0.005156,169.670159,4.595021,9.422957
16,Identity,NaN,0.922775,0.002523,0.970205,0.005267,0.936366,0.001824,0.954771,0.007138,...,0.000878,0.544419,0.006578,0.044419,0.006578,0.916969,0.007839,0.000000,0.000000,inf
17,NegGrad,0.00001,0.922812,0.002419,0.970205,0.005267,0.936346,0.001843,0.954837,0.006944,...,0.000878,0.542016,0.005701,0.042016,0.005701,0.919390,0.006910,0.333553,0.009035,4793.227463
21,Scrub,0.001,0.919006,0.005136,0.944134,0.000000,0.929357,0.006294,0.971873,0.012296,...,0.006145,0.529380,0.001096,0.029380,0.001096,0.930123,0.003664,65.993787,1.643625,24.226441


In [141]:
import pandas as pd

def highlight_best_and_second_best(df, column, ascending=True):
    filtered_df = df[~df['unlearner'].isin(['GoldModel', 'Identity'])]
    sorted_df = filtered_df.sort_values(by=column, ascending=ascending)
    best_value = sorted_df.iloc[0][column]
    second_best_value = sorted_df.iloc[1][column]
    return best_value, second_best_value

def format_value(value, best_value, second_best_value):
    if value == best_value:
        return f"\\textbf{{{value:.3f}}}"
    elif value == second_best_value:
        return f"\\underline{{{value:.3f}}}"
    else:
        return f"{value:.3f}"

def df_to_latex_table(df):
    # Define the LaTeX table header
    header = r"""\begin{table*}[]
\centering
\resizebox{\linewidth}{!}{
\begin{tabular}{l c c c c | c c | c c c | c | c}
\midrule\hline 

DS & Unlearner & Test Acc \up & Forget Acc \up & Retain Acc \up & AUS \up & AIN \down & Forget Diff \down & UMIA \down & Nomus \up & Time (Speedup) \down\\
\midrule\hline \\ 
"""

    # Get the best and second best values for each column
    best_test_acc, second_best_test_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.test.unlearned', ascending=False)
    best_forget_acc, second_best_forget_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.forget.unlearned', ascending=False)
    best_retain_acc, second_best_retain_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.retain.unlearned', ascending=False)
    best_aus, second_best_aus = highlight_best_and_second_best(df, 'AUS', ascending=False)
    best_ain, second_best_ain = highlight_best_and_second_best(df, 'AIN', ascending=True)
    best_forget_diff, second_best_forget_diff = highlight_best_and_second_best(df, 'forget_diff', ascending=True)
    best_umia, second_best_umia = highlight_best_and_second_best(df, 'UMIA', ascending=True)
    best_nomus, second_best_nomus = highlight_best_and_second_best(df, 'nomus', ascending=False)
    best_time, second_best_time = highlight_best_and_second_best(df, 'time', ascending=True)

    # Format the data into LaTeX table rows
    rows = []
    for _, row in df.iterrows():
        # Convert values into "mean ± std" format
        test_acc = f"{format_value(row['sklearn.metrics.accuracy_score.test.unlearned'], best_test_acc, second_best_test_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.test.unlearned_std']:.3f}}}"
        forget_acc = f"{format_value(row['sklearn.metrics.accuracy_score.forget.unlearned'], best_forget_acc, second_best_forget_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.forget.unlearned_std']:.3f}}}"
        retain_acc = f"{format_value(row['sklearn.metrics.accuracy_score.retain.unlearned'], best_retain_acc, second_best_retain_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.retain.unlearned_std']:.3f}}}"
        aus = f"{format_value(row['AUS'], best_aus, second_best_aus)} ± \scriptsize{{{row['AUS_std']:.3f}}}"
        ain = f"{format_value(row['AIN'], best_ain, second_best_ain)} ± \scriptsize{{{row['AIN_std']:.3f}}}"
        forget_diff = f"{format_value(row['forget_diff'], best_forget_diff, second_best_forget_diff)} ± \scriptsize{{{row['forget_diff_std']:.3f}}}"
        umia = f"{format_value(row['UMIA'], best_umia, second_best_umia)} ± \scriptsize{{{row['UMIA_std']:.3f}}}"
        nomus = f"{format_value(row['nomus'], best_nomus, second_best_nomus)} ± \scriptsize{{{row['nomus_std']:.3f}}}"
        time = f"{format_value(row['time'], best_time, second_best_time)} ± \scriptsize{{{row['time_std']:.1f}}}"
        speedup = f"{row['speedup']:.1f}"
        lr = row['lr']

        # Format the row
        rows.append(f"& {row['unlearner']} & {test_acc} & {forget_acc} & {retain_acc} & {aus} & {ain} & {forget_diff} & {umia} & {nomus} & {time} ({speedup}) \\\\")

    # Define the LaTeX table footer
    footer = r"""\midrule\hline 

\end{tabular}}
\caption{Experimental results of different unlearning methods.}
\label{tab:unlearning_results}
\end{table*}"""

    # Combine header, rows, and footer
    latex_table = "\n".join([header] + rows + [footer])

    return latex_table

best_df['unlearner'] = pd.Categorical(best_df['unlearner'], ['Finetuning', 'SuccessiveRandomLabels', 'cfk', 'eu_k', 'NegGrad', 'AdvancedNegGrad', 'unsir', 'BadTeaching', 'Scrub', 'FisherForgetting', 'ssd', 'salun', 'Identity', 'GoldModel'])

best_df = best_df.sort_values('unlearner')

latex_code = df_to_latex_table(best_df)

print(latex_code)  # Print LaTeX output

\begin{table*}[]
\centering
\resizebox{\linewidth}{!}{
\begin{tabular}{l c c c c | c c | c c c | c | c}
\midrule\hline 

DS & Unlearner & Test Acc \up & Forget Acc \up & Retain Acc \up & AUS \up & AIN \down & Forget Diff \down & UMIA \down & Nomus \up & Time (Speedup) \down\\
\midrule\hline \\ 

& Finetuning & \underline{0.924} ± \scriptsize{0.000} & 0.955 ± \scriptsize{0.004} & \textbf{0.950} ± \scriptsize{0.001} & 0.972 ± \scriptsize{0.000} & 1.000 ± \scriptsize{0.000} & 0.019 ± \scriptsize{0.003} & 0.530 ± \scriptsize{0.002} & 0.932 ± \scriptsize{0.002} & 43.586 ± \scriptsize{0.4} (36.7) \\
& SuccessiveRandomLabels & 0.922 ± \scriptsize{0.000} & 0.940 ± \scriptsize{0.001} & 0.925 ± \scriptsize{0.000} & 0.981 ± \scriptsize{0.001} & 1.387 ± \scriptsize{0.078} & \textbf{0.004} ± \scriptsize{0.005} & \textbf{0.504} ± \scriptsize{0.001} & \textbf{0.957} ± \scriptsize{0.001} & 40.439 ± \scriptsize{0.3} (39.5) \\
& cfk & 0.924 ± \scriptsize{0.002} & 0.968 ± \scriptsize{0.009} & 0.938 ± \sc

In [5]:
# import a bert tokenizer 
from transformers import BertTokenizer
import torch 

input = torch.tensor([[2.4, 1.2], [1.2, 3.4]])
print(input.shape)
input_attention_mask = torch.tensor([1, 1])

# create a model 
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-uncased')

output = model(input, attention_mask=input_attention_mask)


torch.Size([2, 2])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)